# Load the Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RNN, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from keras import backend as K

# Define Confusion Matrix

In [4]:
tf.compat.v1.get_default_graph()

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# Load and Prepare the Data

In [ ]:
Train_Data = pd.read_excel(r"Path of Excel file",sheet_name='Adress the Train Sheet')
Test_Data = pd.read_excel(r"Path of Excel file",sheet_name='Adress the Test Sheet')

In [6]:
Predict_day=7 # This algorithm predicts for upcoming 7 days, this value can be changed based on your requirement
MAX_Day=Train_Data.iloc[:,1:2].size # size of training dataset 
MAX_test=Test_Data.iloc[:,1:2].size # size of test dataset

Test_df = Test_Data.iloc[:,1:2].values # Create Test Dataset 
Train_df= Train_Data.iloc[:,1:2].values # Create Train Dataset
sc = MinMaxScaler() #Transform features by scaling each feature to a given range.
RNN_input_Train_data = sc.fit_transform(Train_df) # Fit to data, then transform it.
RNN_input_Test_data = sc.fit_transform(Test_df)

X_train = RNN_input_Train_data[0:(MAX_Day-Predict_day)] # Provide Training Part by considering the Prediction day
Y_train = RNN_input_Train_data[Predict_day:MAX_Day] # Provide Target Part by considering the Prediction day

X_test = Test_df[0:(MAX_test-Predict_day)] # Provide Testing Part by considering the Prediction day
Y_test = Test_df[Predict_day:MAX_test] # Provide Target Part by considering the Prediction day

X_train = np.reshape(X_train,((MAX_Day-Predict_day),1,1))
X_test = np.reshape(X_test,((MAX_test-Predict_day),1,1))


# Different types of "Activation function", "Loss Function", and "Optimizer" Matrix

In [7]:
# Different types of Activation Function
activation_function=['sigmoid','relu','tanh','softmax']
# Different types of Optimizer
Optimizer_=['sgd','RMSprop','Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# Different types of Loss Function
Loos_function=['squared_hinge','hinge','categorical_hinge','logcosh','mean_absolute_error','mean_squared_error','mean_absolute_percentage_error','mean_squared_logarithmic_error']

RMSE=['Activation function','optimizer','RMSE']
evalution=pd.DataFrame([['Activation function','optimizer','ERROR','loss', 'accuracy', 'f1_score', 'precision', 'recall']])

# To evaluate each "Activation function", "Loss Function", and "Optimizer" have the best result, all these parameters have been defined and fitted to the RNN to compare the results

In [ ]:
for j in Optimizer_: #Try to test all Optimizers and find the best
    for i in activation_function: #Try to test all Activation Functions and find the best
        for k in Loos_function: #Try to test all Loss Functions and find the best
            # Define the Network
            RNN_Model = Sequential()
            RNN_Model.add(LSTM(units=45,activation=i,input_shape=(None,1)))
            RNN_Model.add(Dropout(0.1))
            RNN_Model.add(Dense(units=1))
            RNN_Model.compile(optimizer = j, loss = k, metrics=['accuracy', f1_m,precision_m, recall_m])
            History=RNN_Model.fit(X_train, Y_train, batch_size = 30, epochs = 100) 
            
            # Training Process
            inputs = Test_df
            inputs = sc.transform(inputs)
            inputs = np.reshape(inputs, (MAX_test, 1, 1))
            predicted = RNN_Model.predict(inputs)
            predicted = sc.inverse_transform(predicted)
            
            # Plot the Results
            RNN_Model.predict_generator
            plt.plot(Test_df, color = 'red', label = 'Real ')
            plt.plot(predicted, color = 'blue', label = 'Predicted ')
            title_='Activation function= '+i+'   optimizer='+j+'  Loos_Function= '+k
            plt.title(title_)
            plt.xlabel('Time')
            plt.ylabel('Establish Success Rate')
            plt.legend()
            fig = plt.gcf()
            fig.set_size_inches(10, 6)
            plt.show()
            rmse = math.sqrt(mean_squared_error(Test_df, predicted))
            RMSE.append([i,j,rmse])
            print ("Error: "+str(rmse))
            evalu=pd.DataFrame([[i, j, k, rmse, History.history['loss'][epoch], History.history['accuracy'][epoch], History.history['f1_m'][epoch], History.history['precision_m'][epoch], History.history['recall_m'][epoch]]])
            evalution=evalution.append(evalu)

        print(evalu)
